### Import Libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [4]:
dataset = load_dataset("Hello-SimpleAI/HC3", name='all')
dataset = dataset['train'].to_pandas()

Generating train split: 100%|██████████| 24322/24322 [00:00<00:00, 218989.06 examples/s]


In [10]:
dataset.head()

,id,question,human_answers,chatgpt_answers,source
0,0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,reddit_eli5
1,1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,reddit_eli5
2,2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,reddit_eli5
3,3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,reddit_eli5
4,4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,reddit_eli5


In [9]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24322 entries, 0 to 24321
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               24322 non-null  object
 1   question         24322 non-null  object
 2   human_answers    24322 non-null  object
 3   chatgpt_answers  24322 non-null  object
 4   source           24322 non-null  object
dtypes: object(5)
memory usage: 950.2+ KB


### Preprocessing
Notes/Steps:
1. Create df with columns Id, Prompt, Essay, Generated(0,1)
2. Use 1000 essays at first, 500 prompts x 2 essays/prompt (one human, one generated)
3. Split 80/10/10 Train/Test/Validation

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
reduced_df = dataset.head(500).drop(columns=['source'])

In [28]:
human_df = reduced_df[['human_answers']].rename(columns={'human_answers':'text'})
human_df['prompt_id'] = reduced_df["id"]
human_df['text'] = human_df['text'].apply(lambda x: ''.join(x))
human_df['label'] = 0

llm_df = reduced_df[['chatgpt_answers']].rename(columns={'chatgpt_answers':'text'})
llm_df['prompt_id'] = reduced_df["id"]
llm_df['text'] = llm_df['text'].apply(lambda x: ''.join(x))
llm_df['label'] = 1

full_df = pd.concat((human_df, llm_df), axis=0, ignore_index=True)
full_df = full_df[['prompt_id', 'text', 'label']]

In [29]:
full_df

,prompt_id,text,label
0,0,"Basically there are many categories of "" Best ...",0
1,1,salt is good for not dying in car crashes and ...,0
2,2,The way it works is that old TV stations got a...,0
3,3,You ca n't just go around assassinating the le...,0
4,4,Wanting to kill the shit out of Germans drives...,0
...,...,...,...
995,495,"When people look at a map of the Earth, they u...",1
996,496,Sigmund Freud was a psychologist who developed...,1
997,497,"In the United States, the Americans with Disab...",1
998,498,Genetic makeup can make a person more or less ...,1


In [30]:
train_df, temp_df = train_test_split(full_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)